In [1]:
# API
import requests
import json

# AquaCrop
from aquacrop import AquaCropModel, Soil, Crop, InitialWaterContent, IrrigationManagement

# General
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

### Prepare Input Data

In [20]:
# Get all data
refEt = pd.read_csv("fourierEt.csv")
weather = pd.read_csv("weather.csv")

# Separate year 2021 for model identification
refEt21 = refEt[-365*2:-365]
weather21 = weather[-365*2:-365]

# Build weather dataframe for AquaCrop
pd.set_option('chained_assignment', None) # Disable random warning message

weather21.loc[:, "refEt"] = list(refEt21.et)
weather21 = weather21.loc[:, ("temp_min", "temp_max", "rain", "refEt", "date")]
weather21.columns = ["MinTemp", "MaxTemp", "Precipitation", "ReferenceET", "Date"]
weather21["Date"] = pd.to_datetime(weather21["Date"], format= "%Y-%m-%d")
weather21 = weather21.reset_index(drop = True)
weather21

MinTemp  MaxTemp  Precipitation  ReferenceET       Date
0        7.9     11.1            0.1         1.85 2021-01-01
1        5.1     10.2            0.0         1.73 2021-01-02
2        5.1     11.7            0.0         1.94 2021-01-03
3        5.7     12.2            0.0         1.71 2021-01-04
4        6.2     12.4            0.9         1.42 2021-01-05
..       ...      ...            ...          ...        ...
360     15.6     18.7            0.0         1.25 2021-12-27
361     13.5     17.0            0.0         1.83 2021-12-28
362     12.0     20.7            0.0         1.92 2021-12-29
363     12.6     20.5            0.0         1.83 2021-12-30
364      9.3     19.5            0.0         1.71 2021-12-31

[365 rows x 5 columns]

### Define state space
- **RZSMD**: root zone soil moisture deficit (mm)
- **D**: current timestep RZSMD (mm)
- **D+**: next timestep RZSMD (mm)
- **E**: current timestep Evapotranspiration (mm)
- **I**: current timestep Irrigation (mm)
- **P**: current timestep Rainfall (mm)

$$
D^+=c_1D + c_2E + c_3P + c_4I
$$

*deep percolation and water runoff are neglected

The coefficients $c_1$ to $c_4$ need to be identified as they define the contribution to current timestep's RZSMD from the previous day, the correction factor on calculated Evapotranspiration, the effectivness of Rainfall, and the efficiency of Irrigation. To identify them we run an AquaCrop model with data from the previous season based on which we get the value of RZSMD for each day. Using these values we find the coefficients $c_{1:4}$ that minimize the squared deviations between RZSMD data and our state-space predictions.

### Define AquaCrop Model

In [ ]:
# Define model parameters
soil = Soil(soil_type = 'Loam')
crop = Crop('Tomato', planting_date = '04/01', harvest_date = '07/18')
init_water = InitialWaterContent(value = ['FC'])
weather_df = weather21
sim_start_time = weather21.Date.iloc[0].strftime("%Y/%m/%d")
sim_end_time = weather21.Date.iloc[-1].strftime("%Y/%m/%d")

# Define Irrigation Management method
interval_irri = IrrigationManagement(irrigation_method = 2, IrrInterval = 2)

# Initialize field model
model = AquaCropModel(sim_start_time = sim_start_time,
                      sim_end_time = sim_end_time,
                      weather_df = weather_df,
                      soil = soil,
                      crop = crop,
                      initial_water_content = init_water, 
                      irrigation_management = interval_irri)